In [25]:
from db_provider import Provider

from utils import get_inputs_sequence
import matplotlib.pyplot as plt
%matplotlib inline

provider = Provider("database/research_db.sqlite3")

subjects = provider.get_all_subjects()
inputs_sequence_dict = dict()

for subject in subjects:
    inputs_sequence_dict[subject] = get_inputs_sequence(
        provider.get_all_units_by_subject_id(
            subject
        ), is_raw=True
    )

In [26]:
from chapter_one.utils import get_inner_distance_sequence_from_messages_sequence

inner_distance_dict = dict()

for subject in subjects:
    inner_distance_dict[subject] = get_inner_distance_sequence_from_messages_sequence(
        provider.get_messages_by_subject_id(
            subject
        )
    )

In [27]:
frequency_taps = dict()              # общие частоты нажатий клавишь {{key:frequency}}
for subject in subjects:
    for input_item in  inputs_sequence_dict[subject]:
        if input_item.key_code in frequency_taps.keys():
            frequency_taps[input_item.key_code] += 1
        else:
            frequency_taps[input_item.key_code] = 1

taps_count = float( sum( frequency_taps.values() ) )
for key_code in frequency_taps.keys():
    frequency_taps[key_code] = float(frequency_taps[key_code])/taps_count

In [28]:
frequency_taps_for_subjects = dict() # частоты нажатий клавишь для каждого субъекта {subj:{key:frequency}}
for subject in subjects:
    frequency_taps_for_subjects[subject] = dict()
    for input_item in  inputs_sequence_dict[subject]:
        if input_item.key_code in frequency_taps_for_subjects[subject].keys():
            frequency_taps_for_subjects[subject][input_item.key_code] += 1
        else:
            frequency_taps_for_subjects[subject][input_item.key_code] = 1

for subject in subjects:
    taps_count = float(len(inputs_sequence_dict[subject]))
    for key_code in frequency_taps_for_subjects[subject]:
        frequency_taps_for_subjects[subject][key_code] = float(frequency_taps_for_subjects[subject][key_code])/taps_count

In [29]:
most_popular_keys = sorted(frequency_taps, key=lambda k: frequency_taps[k])[-1:-16:-1]
most_active_subjects=[1,3,4]

In [30]:
subject_key_time_dict = dict() #словарь следующего вида {subject: {key: time}}
for subject in most_active_subjects:
    subject_key_time_dict[subject] = dict()
    for ind in inner_distance_dict[subject]:
        if ind.key_code in most_popular_keys:
            if ind.key_code not in subject_key_time_dict[subject].keys():
                subject_key_time_dict[subject][ind.key_code] = list()
            subject_key_time_dict[subject][ind.key_code].append(ind.distance)
min_len = list()
for key in subject_key_time_dict.keys():
    for _key in subject_key_time_dict[key].keys():
        min_len.append(len(subject_key_time_dict[key][_key]))
min_len = min(min_len) # сделал так, потому что могу.
print(min_len)

features = list()
labels = list()
for subject in subject_key_time_dict.keys():
    for i in range(min_len):
        sample = list()
        for key_code in most_popular_keys:
            sample.append(subject_key_time_dict[subject][key_code][i])
        features.append(sample)
        labels.append(subject)

41


In [31]:
from matplotlib import pyplot as plt
%matplotlib inline

import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import KFold

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

features = np.array(features, dtype='float64')
labels = np.array(labels)

classifier = KNeighborsClassifier(n_neighbors=3)
classifier = Pipeline([('norm', StandardScaler()), ('knn', classifier)])

means = list()
kf = KFold(len(features), n_folds=5, shuffle=True)
for training, testing in kf:
    classifier.fit(features[training], labels[training])
    prediction = classifier.predict(features[testing])
    means.append( np.mean(prediction==labels[testing]) )
np.mean(means)

0.86266666666666669

In [32]:
from sklearn.svm import SVC

classifier = SVC()
classifier = Pipeline([('norm', StandardScaler()), ('knn', classifier)])

means = list()
kf = KFold(len(features), n_folds=5, shuffle=True)
for training, testing in kf:
    classifier.fit(features[training], labels[training])
    prediction = classifier.predict(features[testing])
    means.append( np.mean(prediction==labels[testing]) )
np.mean(means)

0.95099999999999996

## Теперь задержки типа отпускание нажатие.

In [104]:
from chapter_one.utils import get_outer_distance_sequence_from_messages_sequence
keys_number = 250

outer_distance_sequence_dict = dict()

for subject in subjects:
    outer_distance_sequence_dict[subject] = get_outer_distance_sequence_from_messages_sequence(
        provider.get_messages_by_subject_id(
            subject
        )
    )

In [105]:
frequency_sharing = dict()

for subject in subjects:
    for outer_dist in outer_distance_sequence_dict[subject]:
        id_od = outer_dist.key_code_from * keys_number + outer_dist.key_code_to
        if id_od in frequency_sharing.keys():
            frequency_sharing[id_od] += 1
        else:
            frequency_sharing[id_od] = 1
sharding_count = float(sum(frequency_sharing.values()))
for key in frequency_sharing.keys():
    frequency_sharing[key] = float(frequency_sharing[key])/sharding_count

In [108]:
most_popular_sharding = sorted(frequency_sharing, key=lambda k: frequency_sharing[k])[-1:-6:-1]

In [109]:
subject_sharding_time_dict = dict() #словарь следующего вида {subject: {sharding: time}}
for subject in most_active_subjects:
    subject_sharding_time_dict[subject] = dict()
    for ond in outer_distance_sequence_dict[subject]:
        id_od = ond.key_code_from * keys_number + ond.key_code_to
        if id_od in most_popular_sharding:
            if id_od not in subject_sharding_time_dict[subject].keys():
                subject_sharding_time_dict[subject][id_od] = list()
            subject_sharding_time_dict[subject][id_od].append(ond.distance)
min_len = list()
for key in subject_sharding_time_dict.keys():
    for _key in subject_sharding_time_dict[key].keys():
        min_len.append(len(subject_sharding_time_dict[key][_key]))
min_len = min(min_len) # сделал так, потому что могу.
print(min_len)

features_s = list()
labels_s = list()
for subject in subject_sharding_time_dict.keys():
    for i in range(min_len):
        sample = list()
        for sharding in most_popular_sharding:
            sample.append(subject_sharding_time_dict[subject][sharding][i])
        features_s.append(sample)
        labels_s.append(subject)

4


In [110]:
features_s = np.array(features_s, dtype='float64')
labels_s = np.array(labels_s)

classifier_knn = KNeighborsClassifier(n_neighbors=3)
classifier_knn = Pipeline([('norm', StandardScaler()), ('knn', classifier_knn)])

means_s = list()
kf_s = KFold(len(features_s), n_folds=5, shuffle=True)
for training, testing in kf_s:
    classifier_knn.fit(features_s[training], labels_s[training])
    prediction = classifier_knn.predict(features_s[testing])
    means_s.append( np.mean(prediction==labels_s[testing]) )
print(np.mean(means_s))

0.533333333333


In [111]:
classifier_svc = SVC()
classifier_svc = Pipeline([('norm', StandardScaler()), ('knn', classifier_svc)])

means_s = list()
kf_s = KFold(len(features_s), n_folds=5, shuffle=True)
for training, testing in kf_s:
    classifier_svc.fit(features_s[training], labels_s[training])
    prediction = classifier_svc.predict(features_s[testing])
    means_s.append( np.mean(prediction==labels_s[testing]) )
print(np.mean(means_s))

0.533333333333


In [92]:
from sklearn.ensemble import VotingClassifier
eclf = VotingClassifier(estimators=[('svc', SVC()), ('knn3', KNeighborsClassifier(n_neighbors=3)), ('knn5', KNeighborsClassifier(n_neighbors=5))], voting='hard')

means_s = list()
kf_s = KFold(len(features_s), n_folds=5, shuffle=True)
for training, testing in kf_s:
    eclf.fit(features_s[training], labels_s[training])
    prediction = eclf.predict(features_s[testing])
    means_s.append( np.mean(prediction==labels_s[testing]) )
print(np.mean(means_s))
                                    

0.504679802956


## Диграфы и триграфы

In [112]:
from chapter_two.utils import get_digraphs_sequence, get_threegraphs_sequence

digraph_sequence_dict = dict()

for subject in most_active_subjects:
    digraph_sequence_dict[subject] = get_digraphs_sequence(
        provider.get_messages_by_subject_id(
            subject
        )
    )

In [113]:
features = list()
labels = list()

for subject in digraph_sequence_dict.keys():
    features += digraph_sequence_dict[subject]
    labels += [subject]*len(digraph_sequence_dict[subject])

In [115]:
print(len(features))
features_s = np.array(features, dtype='float64').reshape(-1, 1)
labels_s = np.array(labels).reshape(-1, 1)

classifier_knn = KNeighborsClassifier(n_neighbors=3)
classifier_knn = Pipeline([('norm', StandardScaler()), ('knn', classifier_knn)])

means_s = list()
kf_s = KFold(len(features_s), n_folds=5, shuffle=True)
for training, testing in kf_s:
    classifier_knn.fit(features_s[training], labels_s[training])
    prediction = classifier_knn.predict(features_s[testing])
    means_s.append( np.mean(prediction==labels_s[testing]) )
print(np.mean(means_s))

/usr/local/lib/python3.4/dist-packages/sklearn/pipeline.py:165: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.steps[-1][-1].fit(Xt, y, **fit_params)
/usr/local/lib/python3.4/dist-packages/sklearn/pipeline.py:165: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.steps[-1][-1].fit(Xt, y, **fit_params)
/usr/local/lib/python3.4/dist-packages/sklearn/pipeline.py:165: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.steps[-1][-1].fit(Xt, y, **fit_params)
/usr/local/lib/python3.4/dist-packages/sklearn/pipeline.py:165: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for examp

9239
0.375602415044


/usr/local/lib/python3.4/dist-packages/sklearn/pipeline.py:165: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.steps[-1][-1].fit(Xt, y, **fit_params)


In [116]:
classifier_svc = SVC()
classifier_svc = Pipeline([('norm', StandardScaler()), ('knn', classifier_svc)])

means_s = list()
kf_s = KFold(len(features_s), n_folds=5, shuffle=True)
for training, testing in kf_s:
    classifier_svc.fit(features_s[training], labels_s[training])
    prediction = classifier_svc.predict(features_s[testing])
    means_s.append( np.mean(prediction==labels_s[testing]) )
print(np.mean(means_s))

/usr/local/lib/python3.4/dist-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)
/usr/local/lib/python3.4/dist-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)
/usr/local/lib/python3.4/dist-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)
/usr/local/lib/python3.4/dist-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1

0.424773780905


/usr/local/lib/python3.4/dist-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)


In [119]:
threegraphs_sequence_dict = dict()

for subject in most_active_subjects:
    threegraphs_sequence_dict[subject] = get_threegraphs_sequence(
        provider.get_messages_by_subject_id(
            subject
        )
    )

In [120]:
features = list()
labels = list()

for subject in threegraphs_sequence_dict.keys():
    features += threegraphs_sequence_dict[subject]
    labels += [subject]*len(threegraphs_sequence_dict[subject])

In [121]:
print(len(features))
features_s = np.array(features, dtype='float64').reshape(-1, 1)
labels_s = np.array(labels).reshape(-1, 1)

classifier_knn = KNeighborsClassifier(n_neighbors=3)
classifier_knn = Pipeline([('norm', StandardScaler()), ('knn', classifier_knn)])

means_s = list()
kf_s = KFold(len(features_s), n_folds=5, shuffle=True)
for training, testing in kf_s:
    classifier_knn.fit(features_s[training], labels_s[training])
    prediction = classifier_knn.predict(features_s[testing])
    means_s.append( np.mean(prediction==labels_s[testing]) )
print(np.mean(means_s))

/usr/local/lib/python3.4/dist-packages/sklearn/pipeline.py:165: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.steps[-1][-1].fit(Xt, y, **fit_params)
/usr/local/lib/python3.4/dist-packages/sklearn/pipeline.py:165: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.steps[-1][-1].fit(Xt, y, **fit_params)
/usr/local/lib/python3.4/dist-packages/sklearn/pipeline.py:165: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.steps[-1][-1].fit(Xt, y, **fit_params)
/usr/local/lib/python3.4/dist-packages/sklearn/pipeline.py:165: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for examp

9112
0.379338714359


/usr/local/lib/python3.4/dist-packages/sklearn/pipeline.py:165: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  self.steps[-1][-1].fit(Xt, y, **fit_params)


In [122]:
classifier_svc = SVC()
classifier_svc = Pipeline([('norm', StandardScaler()), ('knn', classifier_svc)])

means_s = list()
kf_s = KFold(len(features_s), n_folds=5, shuffle=True)
for training, testing in kf_s:
    classifier_svc.fit(features_s[training], labels_s[training])
    prediction = classifier_svc.predict(features_s[testing])
    means_s.append( np.mean(prediction==labels_s[testing]) )
print(np.mean(means_s))

/usr/local/lib/python3.4/dist-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)
/usr/local/lib/python3.4/dist-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)
/usr/local/lib/python3.4/dist-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)
/usr/local/lib/python3.4/dist-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1

0.418039336989


/usr/local/lib/python3.4/dist-packages/sklearn/svm/base.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y_ = column_or_1d(y, warn=True)
